In [1]:
!pip install segmentation-models
!pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.79.0 which is incompatible.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
apache-beam 2.44.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.


In [2]:
import sys
sys.path.append('/kaggle/input/helper-scripts')
sys.path.append('/kaggle/input/fetch1')

In [3]:
%env SM_FRAMEWORK=tf.keras
import segmentation_models as sm
import tensorflow as tf
tf.config.run_functions_eagerly(True)

from fetch_data1 import fetch
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from visualise import visualize, denormalize
import os
import cv2

env: SM_FRAMEWORK=tf.keras
Segmentation Models: using `tf.keras` framework.


In [4]:
base_dir = '/kaggle/input/isic-2016/ISIC 2016 for segmentation (total)/ISIC 2016 for segmentation (total)/'
batch_size = 4 
input_size = (224, 224)
num_epochs = 40
shuffle = False
ratio = 1
grp = 0
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]
model_save = '2016_extend_best_model.h5'

callbacks = [ModelCheckpoint('./'+model_save, 
                             monitor = "val_iou_score",
                             save_weights_only=True, 
                             save_best_only=True, 
                             mode='max',
                             initial_value_threshold = 0.0),
            ReduceLROnPlateau(monitor = "val_iou_score",
                             factor = 0.5,
                             patience = 3,
                             verbose = 1,
                             mode = 'max')]

In [5]:
train_dataset, validation_dataset, val_paths = fetch(base_dir, input_size, grp, batch_size, shuffle, ratio)

train_images:  /kaggle/input/isic-2016/ISIC 2016 for segmentation (total)/ISIC 2016 for segmentation (total)/Train/IMAGES/
no. of training images:  1279
train_masks:  /kaggle/input/isic-2016/ISIC 2016 for segmentation (total)/ISIC 2016 for segmentation (total)/Train/MASKS/
no. of training masks:  1279
test_images:  /kaggle/input/isic-2016/ISIC 2016 for segmentation (total)/ISIC 2016 for segmentation (total)/Test/IMAGES/
no. of test images:  1279
test_masks:  /kaggle/input/isic-2016/ISIC 2016 for segmentation (total)/ISIC 2016 for segmentation (total)/Test/MASKS/
no. of test masks:  1279


/opt/conda/lib/python3.7/site-packages/tensorflow/python/data/ops/structured_function.py:257: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


In [6]:
model = sm.Unet(backbone_name = "densenet201",
                   input_shape=(224, 224, 3))

model.compile(optimizer=Adam(learning_rate = 8e-6), 
              loss=sm.losses.bce_dice_loss, 
              metrics=metrics)

print(len(model.layers))
#model.summary()

74836368/74836368 [==============================] - 0s 0us/step
748


In [7]:
model.load_weights('/kaggle/input/extend-model/2016_extend_best_model.h5') 

In [8]:
'''target_dir = '/kaggle/working/'
from tqdm import tqdm

for image_file in tqdm(os.listdir(base_dir), total=len(os.listdir(base_dir))):
    image_path = os.path.join(base_dir, image_file)
    image = cv2.imread(image_path)  
    print(image.shape)
    mask = model.predict(image).round()
    
    mask_path = os.path.join(target_dir, image_file)
    mask = np.reshape(mask[0], (224, 224))
    if not cv2.imwrite(mask_path, mask):  # Replace with your mask saving code
        raise Exception('Image not saved')'''

"target_dir = '/kaggle/working/'\nfrom tqdm import tqdm\n\nfor image_file in tqdm(os.listdir(base_dir), total=len(os.listdir(base_dir))):\n    image_path = os.path.join(base_dir, image_file)\n    image = cv2.imread(image_path)  \n    print(image.shape)\n    mask = model.predict(image).round()\n    \n    mask_path = os.path.join(target_dir, image_file)\n    mask = np.reshape(mask[0], (224, 224))\n    if not cv2.imwrite(mask_path, mask):  # Replace with your mask saving code\n        raise Exception('Image not saved')"

In [9]:
scores = model.evaluate(validation_dataset)

print("Loss: {:.5}".format(scores[0]))
for metric, value in zip(metrics, scores[1:]):
    print("mean {}: {:.5}".format(metric.__name__, value))

1279/1279 [==============================] - 735s 573ms/step - loss: 0.1371 - iou_score: 0.9121 - f1-score: 0.9518
Loss: 0.13709
mean iou_score: 0.91213
mean f1-score: 0.95177


In [10]:
'''from PIL import Image
import cv2
from tqdm import tqdm
for batch, val_path in tqdm(zip(validation_dataset, val_paths), total=1279):
    img, gt_mask = batch
    img = img.numpy()
    gt_mask = gt_mask.numpy()
    pr_mask = model.predict(img).round()
    
    visualize(
        img=denormalize(img.squeeze()),
        gt_mask=gt_mask[..., 0].squeeze(),
        pr_mask=pr_mask[..., 0].squeeze(),
    )
    pr_mask = np.reshape(pr_mask[0], (224, 224))
    mask = Image.fromarray((pr_mask*255).astype(np.uint8), mode = 'L')
    mask.save('/kaggle/working/'+str(val_path[-16:]))'''

"from PIL import Image\nimport cv2\nfrom tqdm import tqdm\nfor batch, val_path in tqdm(zip(validation_dataset, val_paths), total=1279):\n    img, gt_mask = batch\n    img = img.numpy()\n    gt_mask = gt_mask.numpy()\n    pr_mask = model.predict(img).round()\n    \n    visualize(\n        img=denormalize(img.squeeze()),\n        gt_mask=gt_mask[..., 0].squeeze(),\n        pr_mask=pr_mask[..., 0].squeeze(),\n    )\n    pr_mask = np.reshape(pr_mask[0], (224, 224))\n    mask = Image.fromarray((pr_mask*255).astype(np.uint8), mode = 'L')\n    mask.save('/kaggle/working/'+str(val_path[-16:]))"